# **<font color='red'> Quantum State Reconstruction for pure two-qubit states</font>**
Here, we propose our neural network-based quantum state reconstruction method for pure two-qubit states.

# **<font color='red'> Libraries and Random Seeds</font>**

In [1]:
import numpy as np
import scipy as sp
import scipy.linalg
import matplotlib.pyplot as plt
import qiskit as qk
import qiskit.visualization

from qiskit import Aer
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import Statevector

from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import QuantumError, ReadoutError
from qiskit import transpile
from qiskit.quantum_info.operators import Operator

from qiskit.providers.aer.noise import pauli_error
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise import depolarizing_error
from qiskit.providers.aer.noise import amplitude_damping_error
from qiskit.providers.aer.noise import kraus_error

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
from tensorflow.python.framework.ops import convert_to_tensor

TensorFlow version: 2.11.0


In [2]:
# Ensure reproducibility
np.random.seed(123)
tf.random.set_seed(123)
import random
random.seed(10)

tf.debugging.set_log_device_placement(False)

# **<font color='red'> Functions</font>**

In [3]:
def random_unitary(N):
    """
    Generation of an NxN random unitary matrix.
    """
    Z=np.random.randn(N,N) + 1.0j * np.random.randn(N,N)
    [Q,R]=sp.linalg.qr(Z)
    D=np.diag(np.diagonal(R)/np.abs(np.diagonal(R)))
    return np.dot(Q,D)

In [4]:
def get_density_matrix(state_vector):
    """
    Compute the density matrix from the state vector: 
       rho = |psi >< psi|.
    """
    density_matrix = np.outer(state_vector, np.conjugate(state_vector))
    return density_matrix

In [6]:
# Choice of the simulator
sim_bknd=Aer.get_backend('aer_simulator')

In [8]:
# Generate Haar distributed data

def generate_Haar_data(num_qubits, samples=1000):
    data = []
    for i in range(samples):
        qc = qk.QuantumCircuit(num_qubits) # create a quantum circuit with "num_qubits" qubits
        u = random_unitary(2**num_qubits)
        qc.unitary(u, qubits=range(num_qubits)) # apply the random unitary transformation to the circuit
        qc = qk.transpile(qc, backend=sim_bknd) # optimize the circuit
        qc.save_statevector() # save the state vector obtained by the simulation

        state = sim_bknd.run(qc).result().get_statevector(qc) # simulate and get the state vector
        state = np.asarray(state)
        data.append(state) 
    return data

In [9]:
# Define the two-qubit Pauli basis
pauli_basis1q = np.array([I, X, Y, Z])
pauli_basis2q = np.array([np.kron(a,b) for a in pauli_basis1q for b in pauli_basis1q])
pauli_basis2q_modified = pauli_basis2q[1:]

# Compute the Bloch components from the density matrix for two-qubit states
def bloch_coeffs(rho):
    c = []
    for p in pauli_basis2q_modified:
        c.append(np.trace(rho @ p))
    return np.array(c)

In [11]:
# Define the 4x4 Identity matrix 
I_4 = tf.constant([[1.0, 0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0]],
                  shape=(4,4), dtype = tf.complex64)

# Compute the fidelity between generalized Bloch vectors for two-qubit states
def fid_twoq(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk', pauli_basis2q_modified, a)
   el_b = tf.einsum('ijk,mi->mjk', pauli_basis2q_modified, b)
   rho_a = 0.25 *(el_a + I_4)
   rho_b = 0.25 * (el_b +I_4)
   fidelity = tf.linalg.trace(rho_a @ rho_b) 
   return fidelity 

In [31]:
# Our loss function

@tf.function
def infidelity(a,b):
   a = tf.cast(a, tf.complex64)
   b = tf.cast(b, tf.complex64)
   el_a = tf.einsum('ijk,mi->mjk',pauli_basis2q_modified,a) 
   el_b = tf.einsum('ijk,mi->mjk',pauli_basis2q_modified,b) 
   rho_a = 0.25 *(el_a + I_4)
   rho_b = 0.25 * (el_b +I_4)
   fidelity = tf.linalg.trace(rho_a @ rho_b)
   infidelity = 1 - fidelity
   return infidelity 

# **<font color='red'> Noise Models</font>**

In [15]:
# Probabilities for bit, phase, bit-phase flip or depolarizing channels
p_error = 0.2
num_qubits = 1 # for depolarizing channel
    
# Probabilities for Generalized Pauli channel
p_0 = 0.7
p_1 = 0.2
p_2 = 0.05
p_3 = 0.05

# Channels
bit_flip_error = pauli_error([('X', p_error), ('I', 1 - p_error)])
phase_flip_error = pauli_error([('Z', p_error), ('I', 1 - p_error)])
bit_phase_flip_error = pauli_error([('Y', p_error), ('I', 1 - p_error)])
dep_error = depolarizing_error(p_error, num_qubits)
general_pauli_error = pauli_error([('X', p_1), ('Y', p_2), ('Z', p_3), ('I', p_0)])
AD_error = amplitude_damping_error (0.3, 0.5) 

In [16]:
# apply NOISE whenever an identity gate appears in the circuit (compatibly with the channels parameters)
def qsk_add_error(error_type):
    if error_type == 'bit':
        noise_model.add_all_qubit_quantum_error(bit_flip_error, ["id"])
    if error_type == 'phase':
        noise_model.add_all_qubit_quantum_error(phase_flip_error, ["id"])
    if error_type == 'bit-phase':
        noise_model.add_all_qubit_quantum_error(bit_phase_flip_error, ["id"])
    if error_type == 'depolarizing':
        noise_model.add_all_qubit_quantum_error(dep_error, ["id"])
    if error_type == 'Gpauli':
        noise_model.add_all_qubit_quantum_error(general_pauli_error, ["id"])
    if error_type == 'AD':
        noise_model.add_all_qubit_quantum_error(AD_error, ["id"])
    if error_type == 'Kraus':
        noise_model.add_all_qubit_quantum_error(k_error, ["id"])

#  <font color='red'>Training the Algorithm (MSE)</font>

## <font color='pink'>Different Channels on both Qubits</font>

In [17]:
# Generate data
data = generate_Haar_data(2, 1000) # each state_vector has 4 elements
density_matrix_noise_free = [*map(get_density_matrix, data)] # 16 components for each density matrix

In [18]:
# Apply Noise (here, phase flip on the first qubit and bit flip on the second qubit)
num_qubits = 2
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):

    circ = qk.QuantumCircuit(num_qubits) 

    circ.initialize(data[i])
    circ = transpile(circ, backend=Aer.get_backend('aer_simulator_density_matrix')) 
    circ.barrier()
    circ.id(0) #applies identity to the first qubit
    circ.id(1) #applies identity to the second qubit
    circ.save_density_matrix()
    #print(circ)
    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0]) # add phase flip channel on the first qubit
    noise_model.add_quantum_error(bit_flip_error, ["id"], [1]) # add bit flip channel on the second qubit
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [19]:
vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noise-free Bloch vectors
vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noisy Bloch vectors

In [21]:
# Generate the training, validation and test set

x_train_list = vectors_with_noise[:300]
y_train_list = vectors_noise_free[:300]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = vectors_with_noise[500:800]
y_val_list = vectors_noise_free[500:800]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = vectors_with_noise[800:]
y_test_list = vectors_noise_free[800:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [22]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(15,)),   
  tf.keras.layers.Dense(64, activation='relu'),                                 
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(15),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(3.0) * tf.math.l2_normalize(x, axis=1))
  ])

In [23]:
# Define Loss
loss_fn = tf.keras.losses.mse
# Compile model
adam_opt = tf.optimizers.Adam(0.001)
model.compile(optimizer=adam_opt,
              loss=loss_fn)

In [24]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=50, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 60ms/step - loss: 0.3990 - val_loss: 0.3495
Epoch 2/600
6/6 [==============================] - 0s 11ms/step - loss: 0.3319 - val_loss: 0.2964
Epoch 3/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2756 - val_loss: 0.2456
Epoch 4/600
6/6 [==============================] - 0s 12ms/step - loss: 0.2230 - val_loss: 0.1998
Epoch 5/600
6/6 [==============================] - 0s 10ms/step - loss: 0.1770 - val_loss: 0.1641
Epoch 6/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1434 - val_loss: 0.1383
Epoch 7/600
6/6 [==============================] - 0s 12ms/step - loss: 0.1183 - val_loss: 0.1179
Epoch 8/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0989 - val_loss: 0.1010
Epoch 9/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0831 - val_loss: 0.0875
Epoch 10/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0707 - val_loss: 0.0772
Epoch 11/600
6/6 [=

Epoch 84/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0028 - val_loss: 0.0071
Epoch 85/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0028 - val_loss: 0.0070
Epoch 86/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0027 - val_loss: 0.0070
Epoch 87/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0027 - val_loss: 0.0068
Epoch 88/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0027 - val_loss: 0.0068
Epoch 89/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0026 - val_loss: 0.0067
Epoch 90/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0026 - val_loss: 0.0067
Epoch 91/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0025 - val_loss: 0.0066
Epoch 92/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0025 - val_loss: 0.0066
Epoch 93/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0025 - val_loss: 0.0065
Epoch 94/6

6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0044
Epoch 167/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 168/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 169/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 170/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 171/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 172/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 173/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 174/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0043
Epoch 175/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0012 - val_loss: 0.0042
Epoch 176/600


Epoch 325/600
6/6 [==============================] - 0s 10ms/step - loss: 5.5090e-04 - val_loss: 0.0031
Epoch 326/600
6/6 [==============================] - 0s 10ms/step - loss: 5.5066e-04 - val_loss: 0.0031
Epoch 327/600
6/6 [==============================] - 0s 10ms/step - loss: 5.4883e-04 - val_loss: 0.0030
Epoch 328/600
6/6 [==============================] - 0s 10ms/step - loss: 5.4297e-04 - val_loss: 0.0030
Epoch 329/600
6/6 [==============================] - 0s 10ms/step - loss: 5.3732e-04 - val_loss: 0.0031
Epoch 330/600
6/6 [==============================] - 0s 10ms/step - loss: 5.3622e-04 - val_loss: 0.0031
Epoch 331/600
6/6 [==============================] - 0s 10ms/step - loss: 5.3509e-04 - val_loss: 0.0030
Epoch 332/600
6/6 [==============================] - 0s 10ms/step - loss: 5.3669e-04 - val_loss: 0.0031
Epoch 333/600
6/6 [==============================] - 0s 10ms/step - loss: 5.3262e-04 - val_loss: 0.0030
Epoch 334/600
6/6 [==============================] - 0s 10ms/ste

Epoch 404/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0441e-04 - val_loss: 0.0028
Epoch 405/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0435e-04 - val_loss: 0.0028
Epoch 406/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0576e-04 - val_loss: 0.0028
Epoch 407/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0880e-04 - val_loss: 0.0028
Epoch 408/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0405e-04 - val_loss: 0.0028
Epoch 409/600
6/6 [==============================] - 0s 11ms/step - loss: 4.0089e-04 - val_loss: 0.0028
Epoch 410/600
6/6 [==============================] - 0s 11ms/step - loss: 4.0087e-04 - val_loss: 0.0028
Epoch 411/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0215e-04 - val_loss: 0.0028
Epoch 412/600
6/6 [==============================] - 0s 10ms/step - loss: 3.9873e-04 - val_loss: 0.0028
Epoch 413/600
6/6 [==============================] - 0s 10ms/ste

Epoch 483/600
6/6 [==============================] - 0s 10ms/step - loss: 3.2889e-04 - val_loss: 0.0027
Epoch 484/600
6/6 [==============================] - 0s 10ms/step - loss: 3.3002e-04 - val_loss: 0.0027
Epoch 485/600
6/6 [==============================] - 0s 11ms/step - loss: 3.3277e-04 - val_loss: 0.0027
Epoch 486/600
6/6 [==============================] - 0s 10ms/step - loss: 3.2909e-04 - val_loss: 0.0027
Epoch 487/600
6/6 [==============================] - 0s 9ms/step - loss: 3.2042e-04 - val_loss: 0.0027
Epoch 488/600
6/6 [==============================] - 0s 10ms/step - loss: 3.2168e-04 - val_loss: 0.0027
Epoch 489/600
6/6 [==============================] - 0s 10ms/step - loss: 3.1873e-04 - val_loss: 0.0027
Epoch 490/600
6/6 [==============================] - 0s 10ms/step - loss: 3.2131e-04 - val_loss: 0.0026
Epoch 491/600
6/6 [==============================] - 0s 10ms/step - loss: 3.1230e-04 - val_loss: 0.0026
Epoch 492/600
6/6 [==============================] - 0s 11ms/step

Epoch 562/600
6/6 [==============================] - 0s 10ms/step - loss: 2.6409e-04 - val_loss: 0.0026
Epoch 563/600
6/6 [==============================] - 0s 10ms/step - loss: 2.6409e-04 - val_loss: 0.0025
Epoch 564/600
6/6 [==============================] - 0s 10ms/step - loss: 2.6341e-04 - val_loss: 0.0025
Epoch 565/600
6/6 [==============================] - 0s 11ms/step - loss: 2.5444e-04 - val_loss: 0.0026
Epoch 566/600
6/6 [==============================] - 0s 11ms/step - loss: 2.5689e-04 - val_loss: 0.0025
Epoch 567/600
6/6 [==============================] - 0s 10ms/step - loss: 2.4561e-04 - val_loss: 0.0025
Epoch 568/600
6/6 [==============================] - 0s 12ms/step - loss: 2.5248e-04 - val_loss: 0.0026
Epoch 569/600
6/6 [==============================] - 0s 11ms/step - loss: 2.5320e-04 - val_loss: 0.0026
Epoch 570/600
6/6 [==============================] - 0s 11ms/step - loss: 2.6120e-04 - val_loss: 0.0025
Epoch 571/600
6/6 [==============================] - 0s 10ms/ste

In [25]:
score = model.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - loss: 0.0025 - 172ms/epoch - 25ms/step


In [28]:
# Save the model predictions in a tensor
y_prediction = model(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_twoq(y_prediction, y_test)

print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9952952861785889-3.434251841238911e-11j)


## <font color='pink'>Same Channels on both Qubits</font>

In [34]:
# Generate data
data = generate_Haar_data(2, 1000) 
density_matrix_noise_free = [*map(get_density_matrix, data)] 

In [35]:
# Apply Noise (phase flip channel on both qubits)
num_qubits = 2
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):

    circ = qk.QuantumCircuit(num_qubits) 

    circ.initialize(data[i])
    circ = transpile(circ, backend=Aer.get_backend('aer_simulator_density_matrix')) 
    circ.barrier()
    circ.id(0) # applies identity to the first qubit
    circ.id(1) # applies identity to the second qubit
    circ.save_density_matrix()

    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix 
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0])
    noise_model.add_quantum_error(phase_flip_error, ["id"], [1])
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [36]:
vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noise-free Bloch vectors
vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noisy Bloch vectors

In [37]:
# Define the training, validation and test set

x_train_list = vectors_with_noise[:300]
y_train_list = vectors_noise_free[:300]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = vectors_with_noise[500:800]
y_val_list = vectors_noise_free[500:800]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = vectors_with_noise[800:]
y_test_list = vectors_noise_free[800:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [38]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(15,)),   
  tf.keras.layers.Dense(64, activation='relu'),                                 
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(15),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(3.0) * tf.math.l2_normalize(x, axis=1))
  ])

In [39]:
# Define Loss
loss_fn = tf.keras.losses.mse
# Compile model
adam_opt = tf.optimizers.Adam(0.001)
model.compile(optimizer=adam_opt,
              loss=loss_fn)

In [40]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=50, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 45ms/step - loss: 0.3844 - val_loss: 0.3524
Epoch 2/600
6/6 [==============================] - 0s 12ms/step - loss: 0.3165 - val_loss: 0.2940
Epoch 3/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2567 - val_loss: 0.2385
Epoch 4/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2051 - val_loss: 0.1940
Epoch 5/600
6/6 [==============================] - 0s 13ms/step - loss: 0.1670 - val_loss: 0.1618
Epoch 6/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1392 - val_loss: 0.1365
Epoch 7/600
6/6 [==============================] - 0s 14ms/step - loss: 0.1162 - val_loss: 0.1154
Epoch 8/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0972 - val_loss: 0.0989
Epoch 9/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0829 - val_loss: 0.0859
Epoch 10/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0706 - val_loss: 0.0751
Epoch 11/600
6/6 [=

Epoch 84/600
6/6 [==============================] - 0s 9ms/step - loss: 0.0026 - val_loss: 0.0070
Epoch 85/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0026 - val_loss: 0.0069
Epoch 86/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0026 - val_loss: 0.0069
Epoch 87/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0025 - val_loss: 0.0068
Epoch 88/600
6/6 [==============================] - 0s 9ms/step - loss: 0.0025 - val_loss: 0.0068
Epoch 89/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0025 - val_loss: 0.0067
Epoch 90/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0024 - val_loss: 0.0067
Epoch 91/600
6/6 [==============================] - 0s 9ms/step - loss: 0.0024 - val_loss: 0.0067
Epoch 92/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0024 - val_loss: 0.0066
Epoch 93/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0023 - val_loss: 0.0065
Epoch 94/600


Epoch 246/600
6/6 [==============================] - 0s 12ms/step - loss: 7.3721e-04 - val_loss: 0.0042
Epoch 247/600
6/6 [==============================] - 0s 10ms/step - loss: 7.3216e-04 - val_loss: 0.0042
Epoch 248/600
6/6 [==============================] - 0s 10ms/step - loss: 7.2197e-04 - val_loss: 0.0042
Epoch 249/600
6/6 [==============================] - 0s 10ms/step - loss: 7.1772e-04 - val_loss: 0.0042
Epoch 250/600
6/6 [==============================] - 0s 11ms/step - loss: 7.2288e-04 - val_loss: 0.0042
Epoch 251/600
6/6 [==============================] - 0s 11ms/step - loss: 7.1954e-04 - val_loss: 0.0042
Epoch 252/600
6/6 [==============================] - 0s 10ms/step - loss: 7.0859e-04 - val_loss: 0.0042
Epoch 253/600
6/6 [==============================] - 0s 10ms/step - loss: 7.0551e-04 - val_loss: 0.0041
Epoch 254/600
6/6 [==============================] - 0s 10ms/step - loss: 7.0760e-04 - val_loss: 0.0041
Epoch 255/600
6/6 [==============================] - 0s 10ms/ste

Epoch 325/600
6/6 [==============================] - 0s 11ms/step - loss: 5.1863e-04 - val_loss: 0.0038
Epoch 326/600
6/6 [==============================] - 0s 10ms/step - loss: 5.1644e-04 - val_loss: 0.0038
Epoch 327/600
6/6 [==============================] - 0s 10ms/step - loss: 5.2633e-04 - val_loss: 0.0038
Epoch 328/600
6/6 [==============================] - 0s 10ms/step - loss: 5.2240e-04 - val_loss: 0.0038
Epoch 329/600
6/6 [==============================] - 0s 10ms/step - loss: 5.1943e-04 - val_loss: 0.0038
Epoch 330/600
6/6 [==============================] - 0s 11ms/step - loss: 5.2370e-04 - val_loss: 0.0038
Epoch 331/600
6/6 [==============================] - 0s 11ms/step - loss: 5.2520e-04 - val_loss: 0.0038
Epoch 332/600
6/6 [==============================] - 0s 10ms/step - loss: 5.2679e-04 - val_loss: 0.0038
Epoch 333/600
6/6 [==============================] - 0s 10ms/step - loss: 5.1030e-04 - val_loss: 0.0038
Epoch 334/600
6/6 [==============================] - 0s 10ms/ste

Epoch 404/600
6/6 [==============================] - 0s 10ms/step - loss: 3.9068e-04 - val_loss: 0.0035
Epoch 405/600
6/6 [==============================] - 0s 9ms/step - loss: 3.7873e-04 - val_loss: 0.0035
Epoch 406/600
6/6 [==============================] - 0s 10ms/step - loss: 3.8692e-04 - val_loss: 0.0035
Epoch 407/600
6/6 [==============================] - 0s 10ms/step - loss: 3.8458e-04 - val_loss: 0.0035
Epoch 408/600
6/6 [==============================] - 0s 11ms/step - loss: 3.9155e-04 - val_loss: 0.0035
Epoch 409/600
6/6 [==============================] - 0s 11ms/step - loss: 3.9038e-04 - val_loss: 0.0035
Epoch 410/600
6/6 [==============================] - 0s 11ms/step - loss: 3.8777e-04 - val_loss: 0.0035
Epoch 411/600
6/6 [==============================] - 0s 11ms/step - loss: 3.8563e-04 - val_loss: 0.0035
Epoch 412/600
6/6 [==============================] - 0s 11ms/step - loss: 3.8876e-04 - val_loss: 0.0035
Epoch 413/600
6/6 [==============================] - 0s 10ms/step

Epoch 483/600
6/6 [==============================] - 0s 10ms/step - loss: 3.1840e-04 - val_loss: 0.0033
Epoch 484/600
6/6 [==============================] - 0s 10ms/step - loss: 3.1265e-04 - val_loss: 0.0033
Epoch 485/600
6/6 [==============================] - 0s 10ms/step - loss: 3.0859e-04 - val_loss: 0.0033
Epoch 486/600
6/6 [==============================] - 0s 11ms/step - loss: 3.0493e-04 - val_loss: 0.0033
Epoch 487/600
6/6 [==============================] - 0s 10ms/step - loss: 2.9545e-04 - val_loss: 0.0033
Epoch 488/600
6/6 [==============================] - 0s 10ms/step - loss: 2.9729e-04 - val_loss: 0.0033
Epoch 489/600
6/6 [==============================] - 0s 9ms/step - loss: 2.9818e-04 - val_loss: 0.0033
Epoch 490/600
6/6 [==============================] - 0s 11ms/step - loss: 2.9779e-04 - val_loss: 0.0033
Epoch 491/600
6/6 [==============================] - 0s 10ms/step - loss: 2.9674e-04 - val_loss: 0.0033
Epoch 492/600
6/6 [==============================] - 0s 10ms/step

Epoch 562/600
6/6 [==============================] - 0s 11ms/step - loss: 2.5631e-04 - val_loss: 0.0032
Epoch 563/600
6/6 [==============================] - 0s 12ms/step - loss: 2.5586e-04 - val_loss: 0.0032
Epoch 564/600
6/6 [==============================] - 0s 13ms/step - loss: 2.5142e-04 - val_loss: 0.0032
Epoch 565/600
6/6 [==============================] - 0s 12ms/step - loss: 2.4216e-04 - val_loss: 0.0032
Epoch 566/600
6/6 [==============================] - 0s 11ms/step - loss: 2.4730e-04 - val_loss: 0.0032
Epoch 567/600
6/6 [==============================] - 0s 12ms/step - loss: 2.4758e-04 - val_loss: 0.0032
Epoch 568/600
6/6 [==============================] - 0s 12ms/step - loss: 2.4816e-04 - val_loss: 0.0032
Epoch 569/600
6/6 [==============================] - 0s 13ms/step - loss: 2.5237e-04 - val_loss: 0.0032
Epoch 570/600
6/6 [==============================] - 0s 12ms/step - loss: 2.5199e-04 - val_loss: 0.0032
Epoch 571/600
6/6 [==============================] - 0s 12ms/ste

In [41]:
score = model.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - loss: 0.0034 - 178ms/epoch - 25ms/step


In [42]:
# Save the model predictions in a tensor
y_prediction = model(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_twoq(y_prediction, y_test)

print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9937106370925903+5.122274132629556e-11j)


## <font color='pink'>One Channel on the first qubit</font>

In [43]:
# Generate data
data = generate_Haar_data(2, 1000) #each state_vector has 4 elements
density_matrix_noise_free = [*map(get_density_matrix, data)] #16 components for each density matrix

In [44]:
# Apply Noise (phase flip channel on the first qubit)
num_qubits = 2
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):

    circ = qk.QuantumCircuit(num_qubits) 

    circ.initialize(data[i])
    circ = transpile(circ, backend=Aer.get_backend('aer_simulator_density_matrix')) 
    circ.barrier()
    circ.id(0) # applies identity to the first qubit
    circ.id(1) # applies identity to the second qubit
    circ.save_density_matrix()

    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix 
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0])
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [45]:
vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noise-free Bloch vectors
vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noisy Bloch vectors

In [46]:
# Generate training, validation and test set

x_train_list = vectors_with_noise[:300]
y_train_list = vectors_noise_free[:300]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = vectors_with_noise[500:800]
y_val_list = vectors_noise_free[500:800]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = vectors_with_noise[800:]
y_test_list = vectors_noise_free[800:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [47]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(15,)),   
  tf.keras.layers.Dense(64, activation='relu'),                                 
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(15),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(3.0) * tf.math.l2_normalize(x, axis=1))
  ])

In [48]:
# Define Loss
loss_fn = tf.keras.losses.mse
# Compile model
adam_opt = tf.optimizers.Adam(0.001)
model.compile(optimizer=adam_opt,
              loss=loss_fn)

In [49]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=50, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 39ms/step - loss: 0.3505 - val_loss: 0.3263
Epoch 2/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2832 - val_loss: 0.2682
Epoch 3/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2249 - val_loss: 0.2149
Epoch 4/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1763 - val_loss: 0.1699
Epoch 5/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1393 - val_loss: 0.1365
Epoch 6/600
6/6 [==============================] - 0s 13ms/step - loss: 0.1117 - val_loss: 0.1112
Epoch 7/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0907 - val_loss: 0.0933
Epoch 8/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0754 - val_loss: 0.0802
Epoch 9/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0638 - val_loss: 0.0704
Epoch 10/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0547 - val_loss: 0.0622
Epoch 11/600
6/6 [=

Epoch 84/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0026 - val_loss: 0.0075
Epoch 85/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0026 - val_loss: 0.0074
Epoch 86/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0025 - val_loss: 0.0073
Epoch 87/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0025 - val_loss: 0.0073
Epoch 88/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0025 - val_loss: 0.0072
Epoch 89/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0024 - val_loss: 0.0072
Epoch 90/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0024 - val_loss: 0.0071
Epoch 91/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0024 - val_loss: 0.0071
Epoch 92/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0023 - val_loss: 0.0070
Epoch 93/600
6/6 [==============================] - 0s 15ms/step - loss: 0.0023 - val_loss: 0.0070
Epoch 94/6

6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0051
Epoch 167/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0051
Epoch 168/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0051
Epoch 169/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0051
Epoch 170/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0050
Epoch 171/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0051
Epoch 172/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0050
Epoch 173/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0050
Epoch 174/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0011 - val_loss: 0.0050
Epoch 175/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0011 - val_loss: 0.0050
Epoch 176/600


Epoch 324/600
6/6 [==============================] - 0s 11ms/step - loss: 4.7083e-04 - val_loss: 0.0040
Epoch 325/600
6/6 [==============================] - 0s 11ms/step - loss: 4.6629e-04 - val_loss: 0.0040
Epoch 326/600
6/6 [==============================] - 0s 10ms/step - loss: 4.7154e-04 - val_loss: 0.0040
Epoch 327/600
6/6 [==============================] - 0s 11ms/step - loss: 4.7145e-04 - val_loss: 0.0040
Epoch 328/600
6/6 [==============================] - 0s 11ms/step - loss: 4.7497e-04 - val_loss: 0.0040
Epoch 329/600
6/6 [==============================] - 0s 11ms/step - loss: 4.6339e-04 - val_loss: 0.0040
Epoch 330/600
6/6 [==============================] - 0s 10ms/step - loss: 4.6012e-04 - val_loss: 0.0040
Epoch 331/600
6/6 [==============================] - 0s 10ms/step - loss: 4.7369e-04 - val_loss: 0.0040
Epoch 332/600
6/6 [==============================] - 0s 11ms/step - loss: 4.7032e-04 - val_loss: 0.0040
Epoch 333/600
6/6 [==============================] - 0s 11ms/ste

Epoch 403/600
6/6 [==============================] - 0s 11ms/step - loss: 3.6461e-04 - val_loss: 0.0038
Epoch 404/600
6/6 [==============================] - 0s 11ms/step - loss: 3.5313e-04 - val_loss: 0.0038
Epoch 405/600
6/6 [==============================] - 0s 11ms/step - loss: 3.5244e-04 - val_loss: 0.0038
Epoch 406/600
6/6 [==============================] - 0s 11ms/step - loss: 3.4470e-04 - val_loss: 0.0038
Epoch 407/600
6/6 [==============================] - 0s 12ms/step - loss: 3.3914e-04 - val_loss: 0.0038
Epoch 408/600
6/6 [==============================] - 0s 11ms/step - loss: 3.4185e-04 - val_loss: 0.0038
Epoch 409/600
6/6 [==============================] - 0s 11ms/step - loss: 3.4257e-04 - val_loss: 0.0038
Epoch 410/600
6/6 [==============================] - 0s 11ms/step - loss: 3.4813e-04 - val_loss: 0.0038
Epoch 411/600
6/6 [==============================] - 0s 12ms/step - loss: 3.3849e-04 - val_loss: 0.0038
Epoch 412/600
6/6 [==============================] - 0s 12ms/ste

Epoch 482/600
6/6 [==============================] - 0s 11ms/step - loss: 2.7340e-04 - val_loss: 0.0037
Epoch 483/600
6/6 [==============================] - 0s 12ms/step - loss: 2.6911e-04 - val_loss: 0.0037
Epoch 484/600
6/6 [==============================] - 0s 12ms/step - loss: 2.7155e-04 - val_loss: 0.0036
Epoch 485/600
6/6 [==============================] - 0s 11ms/step - loss: 2.6388e-04 - val_loss: 0.0037
Epoch 486/600
6/6 [==============================] - 0s 11ms/step - loss: 2.6982e-04 - val_loss: 0.0036
Epoch 487/600
6/6 [==============================] - 0s 11ms/step - loss: 2.7296e-04 - val_loss: 0.0037
Epoch 488/600
6/6 [==============================] - 0s 12ms/step - loss: 2.7272e-04 - val_loss: 0.0037
Epoch 489/600
6/6 [==============================] - 0s 12ms/step - loss: 2.6285e-04 - val_loss: 0.0037
Epoch 490/600
6/6 [==============================] - 0s 12ms/step - loss: 2.6578e-04 - val_loss: 0.0037
Epoch 491/600
6/6 [==============================] - 0s 12ms/ste

Epoch 561/600
6/6 [==============================] - 0s 12ms/step - loss: 2.1318e-04 - val_loss: 0.0036
Epoch 562/600
6/6 [==============================] - 0s 12ms/step - loss: 2.1352e-04 - val_loss: 0.0036
Epoch 563/600
6/6 [==============================] - 0s 12ms/step - loss: 2.1597e-04 - val_loss: 0.0036
Epoch 564/600
6/6 [==============================] - 0s 11ms/step - loss: 2.1617e-04 - val_loss: 0.0035
Epoch 565/600
6/6 [==============================] - 0s 11ms/step - loss: 2.1190e-04 - val_loss: 0.0035
Epoch 566/600
6/6 [==============================] - 0s 12ms/step - loss: 2.0797e-04 - val_loss: 0.0036
Epoch 567/600
6/6 [==============================] - 0s 12ms/step - loss: 2.1575e-04 - val_loss: 0.0036
Epoch 568/600
6/6 [==============================] - 0s 11ms/step - loss: 2.1642e-04 - val_loss: 0.0036
Epoch 569/600
6/6 [==============================] - 0s 12ms/step - loss: 2.1071e-04 - val_loss: 0.0036
Epoch 570/600
6/6 [==============================] - 0s 12ms/ste

In [50]:
score = model.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - loss: 0.0035 - 176ms/epoch - 25ms/step


In [51]:
# Save the model predictions in a tensor
y_prediction = model(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_twoq(y_prediction, y_test)

print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9933454990386963-6.944901598249231e-11j)


# <font color='red'>Training the Algorithm (INFIDELITY)</font>

## <font color='PINK'>Different Channels on both Qubits</font>

In [52]:
# Generate data
data = generate_Haar_data(2, 1000) #each state_vector has 4 elements
density_matrix_noise_free = [*map(get_density_matrix, data)] #16 components for each density matrix

In [53]:
# Apply Noise (phase flip channel on the first qubit and bit flip channel on the second qubit)
num_qubits = 2
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):

    circ = qk.QuantumCircuit(num_qubits) 

    circ.initialize(data[i])
    circ = transpile(circ, backend=Aer.get_backend('aer_simulator_density_matrix')) 
    circ.barrier()
    circ.id(0) # applies identity to the first qubit
    circ.id(1) # applies identity to the second qubit
    circ.save_density_matrix()
    #print(circ)
    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix 
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0])
    noise_model.add_quantum_error(bit_flip_error, ["id"], [1])
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [54]:
vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noise-free Bloch vectors
vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noisy Bloch vectors

In [55]:
# Generate the training, validation and test set

x_train_list = vectors_with_noise[:300]
y_train_list = vectors_noise_free[:300]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = vectors_with_noise[500:800]
y_val_list = vectors_noise_free[500:800]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = vectors_with_noise[800:]
y_test_list = vectors_noise_free[800:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [56]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(15,)),   
  tf.keras.layers.Dense(64, activation='relu'),                                 
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(15),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(3.0) * tf.math.l2_normalize(x, axis=1))
  ])

In [57]:
# Compile model
adam_opt = tf.optimizers.Adam(0.001)
model.compile(optimizer=adam_opt,
              loss=infidelity)

In [58]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=50, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 51ms/step - loss: 0.6548 - val_loss: 0.6185
Epoch 2/600
6/6 [==============================] - 0s 11ms/step - loss: 0.5293 - val_loss: 0.5119
Epoch 3/600
6/6 [==============================] - 0s 11ms/step - loss: 0.4217 - val_loss: 0.4183
Epoch 4/600
6/6 [==============================] - 0s 11ms/step - loss: 0.3376 - val_loss: 0.3440
Epoch 5/600
6/6 [==============================] - 0s 13ms/step - loss: 0.2772 - val_loss: 0.2871
Epoch 6/600
6/6 [==============================] - 0s 12ms/step - loss: 0.2286 - val_loss: 0.2439
Epoch 7/600
6/6 [==============================] - 0s 15ms/step - loss: 0.1916 - val_loss: 0.2066
Epoch 8/600
6/6 [==============================] - 0s 12ms/step - loss: 0.1611 - val_loss: 0.1766
Epoch 9/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1371 - val_loss: 0.1527
Epoch 10/600
6/6 [==============================] - 0s 12ms/step - loss: 0.1169 - val_loss: 0.1336
Epoch 11/600
6/6 [=

Epoch 84/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0048 - val_loss: 0.0130
Epoch 85/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0047 - val_loss: 0.0128
Epoch 86/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0046 - val_loss: 0.0128
Epoch 87/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0046 - val_loss: 0.0126
Epoch 88/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0045 - val_loss: 0.0125
Epoch 89/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0044 - val_loss: 0.0125
Epoch 90/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0044 - val_loss: 0.0123
Epoch 91/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0043 - val_loss: 0.0122
Epoch 92/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0042 - val_loss: 0.0121
Epoch 93/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0042 - val_loss: 0.0120
Epoch 94/6

6/6 [==============================] - 0s 12ms/step - loss: 0.0021 - val_loss: 0.0084
Epoch 167/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0085
Epoch 168/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0084
Epoch 169/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0084
Epoch 170/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0083
Epoch 171/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0084
Epoch 172/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0083
Epoch 173/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0083
Epoch 174/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0019 - val_loss: 0.0082
Epoch 175/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0019 - val_loss: 0.0083
Epoch 176/600


6/6 [==============================] - 0s 11ms/step - loss: 0.0013 - val_loss: 0.0071
Epoch 249/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0013 - val_loss: 0.0071
Epoch 250/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0013 - val_loss: 0.0070
Epoch 251/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0071
Epoch 252/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0070
Epoch 253/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0071
Epoch 254/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0070
Epoch 255/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0071
Epoch 256/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0070
Epoch 257/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0070
Epoch 258/600


Epoch 408/600
6/6 [==============================] - 0s 11ms/step - loss: 7.0286e-04 - val_loss: 0.0060
Epoch 409/600
6/6 [==============================] - 0s 10ms/step - loss: 6.8890e-04 - val_loss: 0.0061
Epoch 410/600
6/6 [==============================] - 0s 12ms/step - loss: 7.0715e-04 - val_loss: 0.0061
Epoch 411/600
6/6 [==============================] - 0s 11ms/step - loss: 7.1092e-04 - val_loss: 0.0060
Epoch 412/600
6/6 [==============================] - 0s 11ms/step - loss: 7.1483e-04 - val_loss: 0.0061
Epoch 413/600
6/6 [==============================] - 0s 11ms/step - loss: 6.9952e-04 - val_loss: 0.0060
Epoch 414/600
6/6 [==============================] - 0s 12ms/step - loss: 6.9343e-04 - val_loss: 0.0060
Epoch 415/600
6/6 [==============================] - 0s 11ms/step - loss: 6.8277e-04 - val_loss: 0.0061
Epoch 416/600
6/6 [==============================] - 0s 11ms/step - loss: 6.8572e-04 - val_loss: 0.0061
Epoch 417/600
6/6 [==============================] - 0s 12ms/ste

Epoch 487/600
6/6 [==============================] - 0s 11ms/step - loss: 5.6977e-04 - val_loss: 0.0058
Epoch 488/600
6/6 [==============================] - 0s 11ms/step - loss: 5.5223e-04 - val_loss: 0.0059
Epoch 489/600
6/6 [==============================] - 0s 10ms/step - loss: 5.5331e-04 - val_loss: 0.0059
Epoch 490/600
6/6 [==============================] - 0s 11ms/step - loss: 5.5983e-04 - val_loss: 0.0058
Epoch 491/600
6/6 [==============================] - 0s 11ms/step - loss: 5.6311e-04 - val_loss: 0.0059
Epoch 492/600
6/6 [==============================] - 0s 10ms/step - loss: 5.5394e-04 - val_loss: 0.0058
Epoch 493/600
6/6 [==============================] - 0s 10ms/step - loss: 5.6382e-04 - val_loss: 0.0058
Epoch 494/600
6/6 [==============================] - 0s 10ms/step - loss: 5.4970e-04 - val_loss: 0.0058
Epoch 495/600
6/6 [==============================] - 0s 11ms/step - loss: 5.4212e-04 - val_loss: 0.0059
Epoch 496/600
6/6 [==============================] - 0s 11ms/ste

Epoch 566/600
6/6 [==============================] - 0s 10ms/step - loss: 4.9215e-04 - val_loss: 0.0058
Epoch 567/600
6/6 [==============================] - 0s 11ms/step - loss: 4.9551e-04 - val_loss: 0.0057
Epoch 568/600
6/6 [==============================] - 0s 11ms/step - loss: 4.7357e-04 - val_loss: 0.0057
Epoch 569/600
6/6 [==============================] - 0s 10ms/step - loss: 4.6848e-04 - val_loss: 0.0057
Epoch 570/600
6/6 [==============================] - 0s 10ms/step - loss: 4.5690e-04 - val_loss: 0.0057
Epoch 571/600
6/6 [==============================] - 0s 11ms/step - loss: 4.6254e-04 - val_loss: 0.0057
Epoch 572/600
6/6 [==============================] - 0s 11ms/step - loss: 4.5197e-04 - val_loss: 0.0057
Epoch 573/600
6/6 [==============================] - 0s 11ms/step - loss: 4.7048e-04 - val_loss: 0.0057
Epoch 574/600
6/6 [==============================] - 0s 11ms/step - loss: 4.6880e-04 - val_loss: 0.0058
Epoch 575/600
6/6 [==============================] - 0s 11ms/ste

In [59]:
score = model.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - loss: 0.0053 - 222ms/epoch - 32ms/step


In [60]:
# Save the model predictions in a tensor
y_prediction = model(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_twoq(y_prediction, y_test)

print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9946945905685425-1.5279510295895804e-11j)


## <font color='pink'>Same Channel on both Qubits</font>

In [61]:
# Generate data
data = generate_Haar_data(2, 1000) #each state_vector has 4 elements
density_matrix_noise_free = [*map(get_density_matrix, data)] #16 components for each density matrix

In [62]:
# Apply Noise (here phase flip channel on both qubits)
num_qubits = 2
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):

    circ = qk.QuantumCircuit(num_qubits) 

    circ.initialize(data[i])
    circ = transpile(circ, backend=Aer.get_backend('aer_simulator_density_matrix')) 
    circ.barrier()
    circ.id(0) # applies identity to the first qubit
    circ.id(1) # applies identity to the second qubit
    circ.save_density_matrix()
    #print(circ)
    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] # simulate the circuit and save the density matrix 
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0])
    noise_model.add_quantum_error(phase_flip_error, ["id"], [1])
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [63]:
vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noise-free Bloch vectors
vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noisy Bloch vectors

In [64]:
# Generate training, validation and test set
x_train_list = vectors_with_noise[:300]
y_train_list = vectors_noise_free[:300]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = vectors_with_noise[500:800]
y_val_list = vectors_noise_free[500:800]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = vectors_with_noise[800:]
y_test_list = vectors_noise_free[800:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [65]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(15,)),   
  tf.keras.layers.Dense(64, activation='relu'),                                 
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(15),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(3.0) * tf.math.l2_normalize(x, axis=1))
  ])

In [66]:
# Compile model
adam_opt = tf.optimizers.Adam(0.001)
model.compile(optimizer=adam_opt,
              loss=infidelity)

In [67]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=50, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 41ms/step - loss: 0.7199 - val_loss: 0.6545
Epoch 2/600
6/6 [==============================] - 0s 11ms/step - loss: 0.6087 - val_loss: 0.5618
Epoch 3/600
6/6 [==============================] - 0s 11ms/step - loss: 0.5038 - val_loss: 0.4655
Epoch 4/600
6/6 [==============================] - 0s 12ms/step - loss: 0.4005 - val_loss: 0.3741
Epoch 5/600
6/6 [==============================] - 0s 11ms/step - loss: 0.3136 - val_loss: 0.3024
Epoch 6/600
6/6 [==============================] - 0s 13ms/step - loss: 0.2514 - val_loss: 0.2495
Epoch 7/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2054 - val_loss: 0.2103
Epoch 8/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1693 - val_loss: 0.1782
Epoch 9/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1425 - val_loss: 0.1522
Epoch 10/600
6/6 [==============================] - 0s 10ms/step - loss: 0.1210 - val_loss: 0.1320
Epoch 11/600
6/6 [=

Epoch 84/600
6/6 [==============================] - 0s 16ms/step - loss: 0.0049 - val_loss: 0.0131
Epoch 85/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0048 - val_loss: 0.0131
Epoch 86/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0047 - val_loss: 0.0129
Epoch 87/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0046 - val_loss: 0.0128
Epoch 88/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0046 - val_loss: 0.0127
Epoch 89/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0046 - val_loss: 0.0126
Epoch 90/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0045 - val_loss: 0.0126
Epoch 91/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0044 - val_loss: 0.0124
Epoch 92/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0043 - val_loss: 0.0124
Epoch 93/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0043 - val_loss: 0.0122
Epoch 94/6

6/6 [==============================] - 0s 11ms/step - loss: 0.0021 - val_loss: 0.0088
Epoch 167/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0021 - val_loss: 0.0088
Epoch 168/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0087
Epoch 169/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0087
Epoch 170/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0087
Epoch 171/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0020 - val_loss: 0.0087
Epoch 172/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0087
Epoch 173/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0020 - val_loss: 0.0086
Epoch 174/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0020 - val_loss: 0.0086
Epoch 175/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0086
Epoch 176/600


6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 249/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 250/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 251/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0076
Epoch 252/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 253/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 254/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 255/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 256/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0074
Epoch 257/600
6/6 [==============================] - 0s 13ms/step - loss: 0.0012 - val_loss: 0.0075
Epoch 258/600


Epoch 329/600
6/6 [==============================] - 0s 11ms/step - loss: 8.6701e-04 - val_loss: 0.0070
Epoch 330/600
6/6 [==============================] - 0s 11ms/step - loss: 8.5755e-04 - val_loss: 0.0070
Epoch 331/600
6/6 [==============================] - 0s 11ms/step - loss: 8.6264e-04 - val_loss: 0.0070
Epoch 332/600
6/6 [==============================] - 0s 12ms/step - loss: 8.5937e-04 - val_loss: 0.0070
Epoch 333/600
6/6 [==============================] - 0s 10ms/step - loss: 8.6006e-04 - val_loss: 0.0070
Epoch 334/600
6/6 [==============================] - 0s 12ms/step - loss: 8.5967e-04 - val_loss: 0.0070
Epoch 335/600
6/6 [==============================] - 0s 12ms/step - loss: 8.5315e-04 - val_loss: 0.0070
Epoch 336/600
6/6 [==============================] - 0s 13ms/step - loss: 8.6449e-04 - val_loss: 0.0070
Epoch 337/600
6/6 [==============================] - 0s 13ms/step - loss: 8.5975e-04 - val_loss: 0.0070
Epoch 338/600
6/6 [==============================] - 0s 13ms/ste

Epoch 408/600
6/6 [==============================] - 0s 10ms/step - loss: 6.4420e-04 - val_loss: 0.0067
Epoch 409/600
6/6 [==============================] - 0s 10ms/step - loss: 6.4662e-04 - val_loss: 0.0067
Epoch 410/600
6/6 [==============================] - 0s 10ms/step - loss: 6.4450e-04 - val_loss: 0.0067
Epoch 411/600
6/6 [==============================] - 0s 10ms/step - loss: 6.4416e-04 - val_loss: 0.0067
Epoch 412/600
6/6 [==============================] - 0s 10ms/step - loss: 6.3029e-04 - val_loss: 0.0067
Epoch 413/600
6/6 [==============================] - 0s 11ms/step - loss: 6.3757e-04 - val_loss: 0.0067
Epoch 414/600
6/6 [==============================] - 0s 10ms/step - loss: 6.4728e-04 - val_loss: 0.0067
Epoch 415/600
6/6 [==============================] - 0s 10ms/step - loss: 6.5642e-04 - val_loss: 0.0067
Epoch 416/600
6/6 [==============================] - 0s 10ms/step - loss: 6.4869e-04 - val_loss: 0.0067
Epoch 417/600
6/6 [==============================] - 0s 10ms/ste

Epoch 487/600
6/6 [==============================] - 0s 11ms/step - loss: 5.0567e-04 - val_loss: 0.0065
Epoch 488/600
6/6 [==============================] - 0s 12ms/step - loss: 4.9880e-04 - val_loss: 0.0065
Epoch 489/600
6/6 [==============================] - 0s 11ms/step - loss: 4.9594e-04 - val_loss: 0.0065
Epoch 490/600
6/6 [==============================] - 0s 11ms/step - loss: 5.0084e-04 - val_loss: 0.0065
Epoch 491/600
6/6 [==============================] - 0s 10ms/step - loss: 4.8868e-04 - val_loss: 0.0065
Epoch 492/600
6/6 [==============================] - 0s 10ms/step - loss: 4.8762e-04 - val_loss: 0.0064
Epoch 493/600
6/6 [==============================] - 0s 10ms/step - loss: 4.8699e-04 - val_loss: 0.0064
Epoch 494/600
6/6 [==============================] - 0s 11ms/step - loss: 4.9450e-04 - val_loss: 0.0065
Epoch 495/600
6/6 [==============================] - 0s 12ms/step - loss: 5.0099e-04 - val_loss: 0.0065
Epoch 496/600
6/6 [==============================] - 0s 13ms/ste

Epoch 566/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0121e-04 - val_loss: 0.0063
Epoch 567/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0314e-04 - val_loss: 0.0063
Epoch 568/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0546e-04 - val_loss: 0.0063
Epoch 569/600
6/6 [==============================] - 0s 10ms/step - loss: 4.0086e-04 - val_loss: 0.0064
Epoch 570/600
6/6 [==============================] - 0s 10ms/step - loss: 3.9508e-04 - val_loss: 0.0064
Epoch 571/600
6/6 [==============================] - 0s 11ms/step - loss: 4.0593e-04 - val_loss: 0.0064
Epoch 572/600
6/6 [==============================] - 0s 11ms/step - loss: 4.1166e-04 - val_loss: 0.0063
Epoch 573/600
6/6 [==============================] - 0s 12ms/step - loss: 4.1729e-04 - val_loss: 0.0064
Epoch 574/600
6/6 [==============================] - 0s 11ms/step - loss: 4.1135e-04 - val_loss: 0.0063
Epoch 575/600
6/6 [==============================] - 0s 10ms/ste

In [68]:
score = model.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - loss: 0.0064 - 198ms/epoch - 28ms/step


In [69]:
# Save the model predictions in a tensor
y_prediction = model(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_twoq(y_prediction, y_test)

print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.9935605525970459-1.557055005763086e-11j)


## <font color='pink'>One Channel on the first qubit</font>

In [70]:
# Generate data
data = generate_Haar_data(2, 1000) #each state_vector has 4 elements
density_matrix_noise_free = [*map(get_density_matrix, data)] #16 components for each density matrix

In [71]:
# Apply Noise (phase flip channel on the first qubit)
num_qubits = 2
den_mat_noise_free = [] # noise-free density matrices
den_mat_with_noise = [] # noisy density matrices


for i in range(len(data)):

    circ = qk.QuantumCircuit(num_qubits) 

    circ.initialize(data[i])
    circ = transpile(circ, backend=Aer.get_backend('aer_simulator_density_matrix')) 
    circ.barrier()
    circ.id(0) #applies identity to the first qubit
    circ.id(1) #applies identity to the second qubit
    circ.save_density_matrix()

    ideal_bknd = AerSimulator()
    res = ideal_bknd.run(circ).result().data()['density_matrix'] #simulates the circuit and saves the density matrix obtained
    den_noise_free = np.asarray(res)
    den_mat_noise_free.append(den_noise_free)


    noise_model = NoiseModel()
    noise_model.add_quantum_error(phase_flip_error, ["id"], [0])
    noisy_bknd = AerSimulator(method = 'density_matrix', noise_model=noise_model)
    res_noise = noisy_bknd.run(circ).result().data()['density_matrix']
    den_with_noise = np.asarray(res_noise)
    den_mat_with_noise.append(den_with_noise)

In [72]:
vectors_noise_free = [*map(bloch_coeffs, den_mat_noise_free)] # noise-free Bloch vectors
vectors_with_noise = [*map(bloch_coeffs, den_mat_with_noise)] # noisy Bloch vectors

In [73]:
# Generate training, validation and test set
x_train_list = vectors_with_noise[:300]
y_train_list = vectors_noise_free[:300]

x_train = tf.convert_to_tensor(x_train_list)
y_train = tf.convert_to_tensor(y_train_list)

x_val_list = vectors_with_noise[500:800]
y_val_list = vectors_noise_free[500:800]

x_val = tf.convert_to_tensor(x_val_list)
y_val = tf.convert_to_tensor(y_val_list)

x_test_list = vectors_with_noise[800:]
y_test_list = vectors_noise_free[800:]

x_test = tf.convert_to_tensor(x_test_list)
y_test = tf.convert_to_tensor(y_test_list)

In [74]:
# Define Model
model = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(input_shape=(15,)),   
  tf.keras.layers.Dense(64, activation='relu'),                                 
  tf.keras.layers.Dense(64, activation='relu'), 
  tf.keras.layers.Dense(15),
  tf.keras.layers.Lambda(lambda x:  tf.math.sqrt(3.0) * tf.math.l2_normalize(x, axis=1))
  ])

In [75]:
# Compile model
adam_opt = tf.optimizers.Adam(0.001)
model.compile(optimizer=adam_opt,
              loss=infidelity)

In [76]:
# Train Model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=50, epochs=600)

Epoch 1/600
6/6 [==============================] - 1s 47ms/step - loss: 0.6695 - val_loss: 0.6178
Epoch 2/600
6/6 [==============================] - 0s 11ms/step - loss: 0.5498 - val_loss: 0.5078
Epoch 3/600
6/6 [==============================] - 0s 11ms/step - loss: 0.4349 - val_loss: 0.3964
Epoch 4/600
6/6 [==============================] - 0s 11ms/step - loss: 0.3304 - val_loss: 0.3096
Epoch 5/600
6/6 [==============================] - 0s 12ms/step - loss: 0.2565 - val_loss: 0.2514
Epoch 6/600
6/6 [==============================] - 0s 11ms/step - loss: 0.2061 - val_loss: 0.2099
Epoch 7/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1700 - val_loss: 0.1783
Epoch 8/600
6/6 [==============================] - 0s 12ms/step - loss: 0.1415 - val_loss: 0.1533
Epoch 9/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1207 - val_loss: 0.1342
Epoch 10/600
6/6 [==============================] - 0s 11ms/step - loss: 0.1042 - val_loss: 0.1192
Epoch 11/600
6/6 [=

Epoch 84/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0047 - val_loss: 0.0135
Epoch 85/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0046 - val_loss: 0.0133
Epoch 86/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0046 - val_loss: 0.0133
Epoch 87/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0045 - val_loss: 0.0131
Epoch 88/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0044 - val_loss: 0.0131
Epoch 89/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0044 - val_loss: 0.0130
Epoch 90/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0043 - val_loss: 0.0129
Epoch 91/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0042 - val_loss: 0.0128
Epoch 92/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0042 - val_loss: 0.0127
Epoch 93/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0041 - val_loss: 0.0126
Epoch 94/6

6/6 [==============================] - 0s 11ms/step - loss: 0.0021 - val_loss: 0.0092
Epoch 167/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0021 - val_loss: 0.0092
Epoch 168/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0091
Epoch 169/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0092
Epoch 170/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0020 - val_loss: 0.0091
Epoch 171/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0020 - val_loss: 0.0091
Epoch 172/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 0.0090
Epoch 173/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0019 - val_loss: 0.0090
Epoch 174/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 0.0090
Epoch 175/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0019 - val_loss: 0.0090
Epoch 176/600


6/6 [==============================] - 0s 11ms/step - loss: 0.0013 - val_loss: 0.0078
Epoch 249/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0013 - val_loss: 0.0078
Epoch 250/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0013 - val_loss: 0.0078
Epoch 251/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0013 - val_loss: 0.0078
Epoch 252/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0013 - val_loss: 0.0077
Epoch 253/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0077
Epoch 254/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0077
Epoch 255/600
6/6 [==============================] - 0s 12ms/step - loss: 0.0012 - val_loss: 0.0077
Epoch 256/600
6/6 [==============================] - 0s 11ms/step - loss: 0.0012 - val_loss: 0.0077
Epoch 257/600
6/6 [==============================] - 0s 10ms/step - loss: 0.0012 - val_loss: 0.0077
Epoch 258/600


6/6 [==============================] - 0s 10ms/step - loss: 8.9064e-04 - val_loss: 0.0070
Epoch 330/600
6/6 [==============================] - 0s 11ms/step - loss: 8.9640e-04 - val_loss: 0.0070
Epoch 331/600
6/6 [==============================] - 0s 11ms/step - loss: 8.8518e-04 - val_loss: 0.0070
Epoch 332/600
6/6 [==============================] - 0s 10ms/step - loss: 8.7050e-04 - val_loss: 0.0070
Epoch 333/600
6/6 [==============================] - 0s 11ms/step - loss: 8.6997e-04 - val_loss: 0.0070
Epoch 334/600
6/6 [==============================] - 0s 10ms/step - loss: 8.6777e-04 - val_loss: 0.0069
Epoch 335/600
6/6 [==============================] - 0s 10ms/step - loss: 8.6932e-04 - val_loss: 0.0069
Epoch 336/600
6/6 [==============================] - 0s 10ms/step - loss: 8.6743e-04 - val_loss: 0.0070
Epoch 337/600
6/6 [==============================] - 0s 10ms/step - loss: 8.7500e-04 - val_loss: 0.0070
Epoch 338/600
6/6 [==============================] - 0s 11ms/step - loss: 8.64

Epoch 408/600
6/6 [==============================] - 0s 10ms/step - loss: 6.7635e-04 - val_loss: 0.0066
Epoch 409/600
6/6 [==============================] - 0s 10ms/step - loss: 6.7668e-04 - val_loss: 0.0067
Epoch 410/600
6/6 [==============================] - 0s 10ms/step - loss: 6.7077e-04 - val_loss: 0.0066
Epoch 411/600
6/6 [==============================] - 0s 10ms/step - loss: 6.6373e-04 - val_loss: 0.0066
Epoch 412/600
6/6 [==============================] - 0s 11ms/step - loss: 6.6160e-04 - val_loss: 0.0066
Epoch 413/600
6/6 [==============================] - 0s 11ms/step - loss: 6.5893e-04 - val_loss: 0.0065
Epoch 414/600
6/6 [==============================] - 0s 10ms/step - loss: 6.6150e-04 - val_loss: 0.0066
Epoch 415/600
6/6 [==============================] - 0s 10ms/step - loss: 6.6195e-04 - val_loss: 0.0066
Epoch 416/600
6/6 [==============================] - 0s 11ms/step - loss: 6.5449e-04 - val_loss: 0.0066
Epoch 417/600
6/6 [==============================] - 0s 11ms/ste

Epoch 487/600
6/6 [==============================] - 0s 10ms/step - loss: 5.4653e-04 - val_loss: 0.0065
Epoch 488/600
6/6 [==============================] - 0s 11ms/step - loss: 5.5941e-04 - val_loss: 0.0064
Epoch 489/600
6/6 [==============================] - 0s 10ms/step - loss: 5.4533e-04 - val_loss: 0.0064
Epoch 490/600
6/6 [==============================] - 0s 10ms/step - loss: 5.3777e-04 - val_loss: 0.0063
Epoch 491/600
6/6 [==============================] - 0s 10ms/step - loss: 5.1640e-04 - val_loss: 0.0063
Epoch 492/600
6/6 [==============================] - 0s 11ms/step - loss: 5.1913e-04 - val_loss: 0.0063
Epoch 493/600
6/6 [==============================] - 0s 10ms/step - loss: 5.2956e-04 - val_loss: 0.0064
Epoch 494/600
6/6 [==============================] - 0s 11ms/step - loss: 5.3422e-04 - val_loss: 0.0063
Epoch 495/600
6/6 [==============================] - 0s 10ms/step - loss: 5.0548e-04 - val_loss: 0.0063
Epoch 496/600
6/6 [==============================] - 0s 10ms/ste

Epoch 566/600
6/6 [==============================] - 0s 11ms/step - loss: 4.2259e-04 - val_loss: 0.0062
Epoch 567/600
6/6 [==============================] - 0s 10ms/step - loss: 4.2132e-04 - val_loss: 0.0062
Epoch 568/600
6/6 [==============================] - 0s 10ms/step - loss: 4.2996e-04 - val_loss: 0.0062
Epoch 569/600
6/6 [==============================] - 0s 11ms/step - loss: 4.2500e-04 - val_loss: 0.0062
Epoch 570/600
6/6 [==============================] - 0s 10ms/step - loss: 4.1727e-04 - val_loss: 0.0062
Epoch 571/600
6/6 [==============================] - 0s 10ms/step - loss: 4.1565e-04 - val_loss: 0.0062
Epoch 572/600
6/6 [==============================] - 0s 10ms/step - loss: 4.1425e-04 - val_loss: 0.0062
Epoch 573/600
6/6 [==============================] - 0s 10ms/step - loss: 4.1638e-04 - val_loss: 0.0062
Epoch 574/600
6/6 [==============================] - 0s 10ms/step - loss: 4.2126e-04 - val_loss: 0.0062
Epoch 575/600
6/6 [==============================] - 0s 10ms/ste

In [77]:
score = model.evaluate(x_test,  y_test, verbose=2)

7/7 - 0s - loss: 0.0055 - 174ms/epoch - 25ms/step


In [78]:
# Save the model predictions in a tensor
y_prediction = model(x_test)
y_prediction = y_prediction.numpy().tolist()

y_test = tf.cast(y_test, dtype=tf.float32)
x_test = tf.cast(x_test, dtype=tf.float32)


fidelities = fid_twoq(y_prediction, y_test)

print(f"Fidelity between ideal and predicted Bloch vectors: {tf.math.reduce_mean(fidelities).numpy()}")

Fidelity between ideal and predicted Bloch vectors: (0.994452953338623-5.587935322792781e-11j)
